<div class="alert alert-block alert-info" style="margin-top: 20px">
    <a href="https://cocl.us/System_ML_notebook">
         <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0111EN/Ad/TopAd.png" width="750" align="center">
    </a>
</div>

<a href="https://cognitiveclass.ai/">
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0111EN/Ad/CCLog.png" width="200" align="center">
</a>

# Declarative Machine Learning

In this exercise, we will look at DML scripts, both in DML and PyDML (shown as examples). This runbook is developed to run with DML. The DML uses R-like syntax whereas PyDML, you guessed it, uses Python-like syntax. These scripts can run on Hadoop, Spark, or in Standalone mode with no modifications required. SystemML automatically performs advanced optimizations based on data and cluster characteristics, so much of the need to manually tweak algorithms is largely reduced or eliminated.

In this lab exercise, we will create a string that represents the content of the actual DML or PyDML scripts. The string will be passed in to the execute function to run. Before we get to that, this is how you would run something in the Spark-shell, as an example.

Suppose that you have this inside your hello.dml and hello.pydml script: `print ('Hello ' + $param1)`

One way to begin working with SystemML is to build the project and unpack the standalone distribution, which features the <b>runStandaloneSystemML.sh</b> and <b>runStandaloneSystemML.bat</b> scripts. The name of the DML or PyDML script is passed as the first argument to these scripts, along with a variety of arguments. Remember the named arguments are recommended over positional arguments, so that is what we are showing here.

`./runStandaloneSystemML.sh hello.dml -nvargs param1="world"` <br>
`./runStandaloneSystemML.sh hello.pydml -python -nvargs param1="world"`

Throughout the rest of this notebook, you will create a variable that contains actual DML code and will execute the variable as if we were executing a script.

First thing is to add the SystemML package.

In [ ]:
%AddDeps org.apache.systemml systemml 1.2.0

Create the variable `hello_dml` to represent the content of a DML script.

In [ ]:
val hello_dml = 
"""
print("Hello " + $param1)
"""

Import the <code>mlcontext</code> and create the named argument variable. Then run the DML script.

In [ ]:
import org.apache.sysml.api.mlcontext._
val ml = new MLContext(sc)      
val nargs = Map("param1" -> "SystemML")
val dml = ScriptFactory.dml(hello_dml).in(nargs)
ml.execute(dml)

Now that we have figured out how to run the variables as DML scripts, we'll develop the script some more with various DML syntax and concepts.

## Variables

DML variable rules:
```
A     # valid variable name
_A    # invalid variable name -- starts with underscore
1_A   # invalid variable name -- starts with number
A_1   # valid variable name 
```

In [ ]:
val variable_example = 
"""
helloStr = "Hello World"
print(helloStr)

helloStr_1 = "Hello World_1"
print (helloStr_1)
"""
val variable_dml = ScriptFactory.dml(variable_example)
ml.execute(variable_dml)

## Data Types

SystemML has four value data types. In DML, these are: <b>double</b>, <b>integer</b>, <b>string</b>, and <b>boolean</b>. In PyDML, these are: <b>float</b>, <b>int</b>, <b>str</b>, and <b>bool</b>. In normal usage, the data type of a variable is implicit based on its value. Mathematical operations typically operate on doubles/floats, whereas integers/ints are typically useful for tasks such as iteration and accessing elements in a matrix.

Declare a datatype variable. Again, this could be an actual dml script that you pass to the MLContext if you were running in a spark shell. 

In [ ]:
val datatype_example =
"""
aDouble = 3.0
bInteger = 2
print('aDouble = ' + aDouble)
print('bInteger = ' + bInteger)
print('aDouble + bInteger = ' + (aDouble + bInteger))
print('bInteger ^ 3 = ' + (bInteger ^ 3))
print('aDouble ^ 2 = ' + (aDouble ^ 2))

cBoolean = TRUE
print('cBoolean = ' + cBoolean)
print('(2 < 1) = ' + (2 < 1))

dString = 'Hello'
eString = dString + ' World'
print('dString = ' + dString)
print('eString = ' + eString)

"""
val datatype_dml = ScriptFactory.dml(datatype_example)
ml.execute(datatype_dml)

Here's another example with a preview of the built-in function matrix. It is use to create a matrix. More will be covered in the next task. Create a matrix of 10 rows by 10 columns with the value 0s for each cell. Define the variable b of value 10 and C as the sum of the matrix A + the value of scalar B.

In [ ]:
val datatype_example2 = 
"""
A = matrix(0, rows=10, cols=10)
B = 10
C = B + sum(A)
print( "B:" + B + ", C:" + C + ", A[1,1]:" + as.scalar(A[1,1]))
"""
val datatype_dml2 = ScriptFactory.dml(datatype_example2)
ml.execute(datatype_dml2)

Value B is as expected. Value C is the value of B added with the value of the row 1, column 1, in matrix A. In this case, the value is zero, so the value of C is 10.0 (a scalar double) due to the matrix addition. The function `as.scalar` converts the value in the cell to a scalar.

## Matrix Basics

### Creating a Matrix

A matrix can be created in DML using the <code>matrix()</code> function and in PyDML using the <code>full()</code> function. In the example below, a matrix element is still considered to be of the matrix data type, so the value is cast to a scalar in order to print it. Matrix element values are of type double/float.

In [ ]:
val matrix_example = 
"""
m = matrix("1 2 3 4 5 6 7 8 9 10 11 12", rows=4, cols=3)
for (i in 1:nrow(m)) {
    for (j in 1:ncol(m)) {
        n = m[i,j]
        print('[' + i + ',' + j + ']:' + as.scalar(n))
    }
}
"""
val matrix_dml = ScriptFactory.dml(matrix_example)
ml.execute(matrix_dml)

The script above creates a matrix of 4 rows by 3 columns with the values from 1 through 12. Then iterate through the matrix and use the `as.scalar(n)` to convert the matrix cell to a scalar double. The PyDML version would use `full()` and `scalar(n)` instead of `matrix()` and `as.scalar(n)`.

You can also print out values of the matrix using the `toString` function.

In [ ]:
val matrix_example2 = 
"""
m = matrix("1 2 3 4 5 6 7 8 9 10 11 12", rows=4, cols=3)
print(toString(m, sep=" | ", decimal=1))
"""

val matrix_dml2 = ScriptFactory.dml(matrix_example2)
ml.execute(matrix_dml2)

## Matrix Operations

DML and PyDML offer a rich set of operators and built-in functions to perform various operations on matrices and scalars. Operators and built-in functions are described in great detail in the Language Reference (Expressions, Built-In Functions).

In this example, we create a matrix A. Next, we create another matrix B by adding 4 to each element in A. Next, we flip B by taking its transpose. We then multiply A and B, represented by matrix C. We create a matrix D with the same number of rows and columns as C, and initialize its elements to 5. We then subtract D from C and divide the values of its elements by 2 and assign the resulting matrix to D.

In [ ]:
val matrix_operations =
"""
A = matrix("1 2 3 4 5 6", rows=3, cols=2)
print("Matrix A:\n" + toString(A))
B = A + 4
B = t(B)
print("Matrix B:\n" + toString(B))
C = A %*% B
print("Matrix C:\n" + toString(C))
D = matrix(5, rows=nrow(C), cols=ncol(C))
D = (C - D) / 2
print("Matrix D:\n" + toString(D))
"""
val matrixop_dml = ScriptFactory.dml(matrix_operations)
ml.execute(matrixop_dml)

## Matrix Indexing

The elements in a matrix can be accessed by their row and column indices. In the example below, we have 3x3 matrix A. First, we access the element at the third row and third column. Next, we obtain a row slice (vector) of the matrix by specifying the row and leaving the column blank. We obtain a column slice (vector) by leaving the row blank and specifying the column. After that, we obtain a submatrix via range indexing, where we specify rows, separated by a colon, and columns, separated by a colon.

In [ ]:
val matrix_indexing = 
"""
A = matrix("1 2 3 4 5 6 7 8 9", rows=3, cols=3)
print("Matrix A: \n" + toString(A))
B = A[3,3]
print("Matrix B: \n" + toString(B))
C = A[2,]
print("Matrix C: \n" + toString(C))
D = A[,3]
print("Matrix D: \n" + toString(D))
E = A[2:3,1:2]
print("Matrix E: \n" + toString(E))
"""
val matrixi_dml = ScriptFactory.dml(matrix_indexing)
ml.execute(matrixi_dml)

## Control Statements

DML and PyDML both feature if, if-else, and if-else-if conditional statements.

DML and PyDML feature 3 loop statements: while, for, and parfor (parallel for). In the example, note that the print statements within the parfor loop can occur in any order since the iterations occur in parallel rather than sequentially as in a regular for loop. The parfor statement can include several optional parameters, as described in the Language Reference (ParFor Statement).

In [ ]:
val control_example = 
"""
i = 1
while (i <= 3) {
    if (i == 1) {
        print('hello')
    } else if (i == 2) {
        print('world')
    } else {
        print('!!!')
    }
    i = i + 1
}

A = matrix("1 2 3 4 5 6", rows=3, cols=2)

for (i in 1:nrow(A)) {
    print("for A[" + i + ",1]:" + as.scalar(A[i,1]))
}

parfor(i in 1:nrow(A)) {
    print("parfor A[" + i + ",1]:" + as.scalar(A[i,1]))
}
"""
val control_dml = ScriptFactory.dml(control_example)
ml.execute(control_dml)

## User-Defined Functions

Functions encapsulate useful functionality in SystemML. In addition to built-in functions, users can define their own functions. Functions take 0 or more parameters and return 0 or more values.

Currently, if a function returns nothing, it still needs to be assigned to a variable.

In this example here, we create a function that performs some matrix operation and writes the results out to the filesystem. In our case, it is written to your DSWB environment. 

In [ ]:
val udf_example = 
"""
doSomething = function(matrix[double] mat) return (matrix[double] ret) {
    additionalCol = matrix(1, rows=nrow(mat), cols=1) # 1x3 matrix with 1 values
    ret = cbind(mat, additionalCol) # concatenate column to matrix
    ret = cbind(ret, seq(0, 2, 1))  # concatenate column (0,1,2) to matrix
    ret = cbind(ret, rowMaxs(ret))  # concatenate column of max row values to matrix
    ret = cbind(ret, rowSums(ret))  # concatenate column of row sums to matrix
}

A = rand(rows=3, cols=2, min=0, max=2) # random 3x2 matrix with values 0 to 2
B = doSomething(A)
write(A, "A.csv", format="csv")
write(B, "B.csv", format="csv")
"""
val udf_dml = ScriptFactory.dml(udf_example)
ml.execute(udf_dml)

In the above example, a 3x2 matrix of random doubles between 0 and 2 is created using the rand() function. Additional parameters can be passed to rand() to control sparsity and other matrix characteristics.

Matrix A is passed to the doSomething function. A column of 1 values is concatenated to the matrix. A column consisting of the values (0, 1, 2) is concatenated to the matrix. Next, a column consisting of the maximum row values is concatenated to the matrix. A column consisting of the row sums is concatenated to the matrix, and this resulting matrix is returned to variable B. Matrix A is output to the A.csv file and matrix B is saved as the B.csv file.

The output of the UDF is written to the filesystem. Check out its content by going to the my-data section of the DSWB and downloading the file. You can also download the file directly from under the Recent Data panel on the Side Bar (you may need to toggle the Side Bar to show it).

<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Get IBM Watson Studio free of charge!</h2>
    <p><a href="https://cocl.us/System_ML_notebook"><img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0111EN/Ad/BottomAd.png" width="750" align="center"></a></p>
</div>